In [16]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
from collections import Counter
from operator import itemgetter
import time
from matplotlib_venn import venn2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline

# magic word for producing visualizations in notebook
%matplotlib inline

### Load Data
Since the dataset is more than 1GB, I've used `c` engine instead of `python` to load data faster. In addition, there are mixed data in column 18 & 19; more specifically some `NaN` values are represented by `X` & `XX`. These values are set as `na_values`.

In [17]:
# Load data
def load_data(azdias_filepath, customers_filepath, attributes_filepath, attributes_desc_filepath):
    """
    Method for loading dataset from CSV & Excel
    
    Args:
        azdias_filepath (str): Azdias filepath (Udacity_AZDIAS_052018)
        customers_filepath (str): Customers filepath (Udacity_CUSTOMERS_052018)
        attributes_filepath (str): Attributes filepath (DIAS Attributes - Values 2017.xlsx)
        attributes_desc_filepath (str): Attributes description (DIAS Information Levels - Attributes 2017.xlsx)
        
    Output:
        azdias: Pandas Dataframe
        customers: Pandas Dataframe
        attributes: Pandas Dataframe
        attributes_desc: Pandas Dataframe
    """
    
    # Load "azdias" dataset
    azdias = pd.read_csv(azdias_filepath, na_values=["X", "XX"], engine="c")
    
    # Load "customers" dataset
    customers = pd.read_csv(customers_filepath, na_values=["X", "XX"], engine="c")
    
    # Load "attributes" dataset
    attributes = pd.read_excel(attributes_filepath, header=1).loc[:, ["Attribute", "Value", "Meaning"]].fillna(method='ffill')
    
    # Load "attributes_desc"
    attributes_desc = pd.read_excel(attributes_desc_filepath, header=1).loc[:, ["Information level", "Attribute", "Description",
                                                                                "Additional notes"]].fillna(method='ffill')
    
    return azdias, customers, attributes, attributes_desc

In [18]:
%%time
azdias_filepath = "Udacity_AZDIAS_052018.csv"
customers_filepath = "Udacity_CUSTOMERS_052018.csv"
attributes_filepath = "DIAS Attributes - Values 2017.xlsx"
attributes_desc_filepath = "DIAS Information Levels - Attributes 2017.xlsx"

azdias, customers, attributes, attributes_desc = load_data(azdias_filepath, customers_filepath, attributes_filepath,
                                                         attributes_desc_filepath)

Wall time: 45.4 s


### Explore the Datasets

In [19]:
print("General population dataset, azdias shape: ", azdias.shape)
print("Customers dataset, customers shape: ", customers.shape)

General population dataset, azdias shape:  (891221, 366)
Customers dataset, customers shape:  (191652, 369)


In [23]:
lnr_azdias = list(azdias.LNR.unique())
lnr_customers = list(customers.LNR.unique())
common = set(lnr_customers) - set(lnr_azdias)
len(common)

191652

In [5]:
# Create a dataframe that describes the information about each features
def build_feat_info(df):
    """
    Method for finding feature informations
    
    Args:
        df (Pandas Dataframe): Dataframe that needs to be described
        
    Output:
        Pandas Dataframe
    """
    return pd.DataFrame({
        'value_count' : [df[x].count() for x in df.columns],
        'value_distinct' : [df[x].unique().shape[0] for x in df.columns],
        'num_nans': [df[x].isnull().sum() for x in df.columns],
        'percent_nans' : [round(df[x].isnull().sum()/df[x].shape[0], 3) for x in df.columns],
    }, index = df.columns)

In [6]:
%%time
feat_info_azdias = build_feat_info(azdias)
feat_info_azdias.sort_values(by=['percent_nans'], ascending=False)

Wall time: 6.87 s


,value_count,value_distinct,num_nans,percent_nans
ALTER_KIND4,1205,13,890016,0.999
ALTER_KIND3,6170,16,885051,0.993
ALTER_KIND2,29499,18,861722,0.967
ALTER_KIND1,81058,18,810163,0.909
EXTSEL992,237068,57,654153,0.734
...,...,...,...,...
D19_VERSAND_ANZ_24,891221,7,0,0.000
D19_VERSAND_DATUM,891221,10,0,0.000
D19_VERSAND_OFFLINE_DATUM,891221,10,0,0.000
D19_VERSAND_ONLINE_DATUM,891221,10,0,0.000


In [7]:
%%time
feat_info_customers = build_feat_info(customers)
feat_info_customers.sort_values(by=['percent_nans'], ascending=False)

Wall time: 1.84 s


,value_count,value_distinct,num_nans,percent_nans
ALTER_KIND4,236,11,191416,0.999
ALTER_KIND3,1275,15,190377,0.993
ALTER_KIND2,5100,18,186552,0.973
ALTER_KIND1,11766,18,179886,0.939
KK_KUNDENTYP,79715,7,111937,0.584
...,...,...,...,...
D19_VERSAND_ANZ_24,191652,7,0,0.000
D19_VERSAND_DATUM,191652,10,0,0.000
D19_VERSAND_OFFLINE_DATUM,191652,10,0,0.000
D19_VERSAND_ONLINE_DATUM,191652,10,0,0.000


### Check Features 
#### Create Features List from `attributes` and `attributes_desc`

In [8]:
# Features listed in "attributes" 
features_attributes = sorted(list(set(attributes.Attribute.tolist())))

# Features listed in attributes_desc 
features_attributes_desc = attributes_desc.Attribute.tolist()

# Multiple features in attributes_desc are grouped in a single string separated by whitespaces
# Split the multiple features into separate string
features_attributes_desc = sorted([feature for group_feature in features_attributes_desc for feature in group_feature.split()])

# Check if there is any unique features in attributes or in attributes_desc
unique_attributes_features = set(features_attributes) - set(features_attributes_desc)
unique_attributes_desc_features = set(features_attributes_desc) - set(features_attributes)
print("Unique attributes features: {}, Unique attributes_desc features: {}".format(len(unique_attributes_features),
                                                                                  len(unique_attributes_desc_features)))

# Combine all features and create the features list
features_list = sorted(list(set.union(set(features_attributes), set(features_attributes_desc))))
print("Total number of features: ", len(features_list))

Unique attributes features: 4, Unique attributes_desc features: 8
Total number of features:  322


#### Check Features of `azdias` and `customers`
Using `features_list` generated earlier, check how many features are present in `azdias` and `customers`

In [9]:
azdias_feat = azdias.columns.tolist()
customers_feat = customers.columns.tolist()

# First check if all the features from features_list are present in azdias and customers
common_azdias_feat = set(azdias_feat) & set(features_list)
common_customers_feat = set(customers_feat) & set(features_list)

print("Number of common features: azdias_feat: {}, customers_feat: {}".format(len(common_azdias_feat),
                                                                              len(common_customers_feat)))

Number of common features: azdias_feat: 276, customers_feat: 276


**Observations:** There are 8 features in `features_list` which are present in `attributes_desc`, but not in `attributes`. These features have descriptions but their encoded values cannot be found in `attributes`. Let's explore these features and their values in `azdias` before further analysis of all the datasets.

In [10]:
unique_attributes_desc_features = list(unique_attributes_desc_features)
azdias_subset = azdias.loc[:, unique_attributes_desc_features]
feat_info_azdias_subset = build_feat_info(azdias_subset)
feat_info_azdias_subset.sort_values(by=['percent_nans'], ascending=False)

C:\Users\Reza\Anaconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,value_count,value_distinct,num_nans,percent_nans
PLZ8,0,1,891221,1.000
EINWOHNER,0,1,891221,1.000
PLZ,0,1,891221,1.000
GKZ,0,1,891221,1.000
ARBEIT,794005,7,97216,0.109
D19_VERSI_OFFLINE_DATUM,891221,10,0,0.000
D19_VERSI_ONLINE_DATUM,891221,10,0,0.000
D19_VERSI_DATUM,891221,10,0,0.000


**Observations:** Features with 100% `NaN` counts can be dropped during the downstream data cleaning process. However, four features: i. `ARBEIT`, ii. `D19_VERSI_DATUM`, iii. `D19_VERSI_OFFLINE_DATUM` iv. `D19_VERSI_ONLINE_DATUM`, have relatively low `NaN` counts. These features should be preserved during data cleaning process. First, let's explore the distinct values for each columns since they are not described in `attributes`.

In [11]:
# Drop the columns with high NaN percentage from azdias_subset
azdias_subset.drop(columns=["EINWOHNER", "PLZ8", "PLZ", "GKZ"], inplace=True)
azdias_subset_distinct_val = pd.DataFrame({
                                "value_list" : [sorted(azdias_subset[x].unique()) for x in azdias_subset.columns]
                            }, index = azdias_subset.columns)
azdias_subset_distinct_val

,value_list
D19_VERSI_OFFLINE_DATUM,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
ARBEIT,"[nan, 1.0, 2.0, 3.0, 4.0, 5.0, 9.0]"
D19_VERSI_ONLINE_DATUM,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
D19_VERSI_DATUM,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"


**Assumption:** Referring to `DIAS Information Levels - Attributes 2017.xlsx`, `ARBEIT` belongs to community level features. Community level features also include `RELAT_AB` whose description is similar to `ARBEIT`. Therefore, value encoded for `RELAT_AB` will be used as reference for `ARBEIT`.

`D19_VERSI_DATUM`, `D19_VERSI_OFFLINE_DATUM`, and `D19_VERSI_ONLINE_DATUM` have no missing values and no encoding description is available for these features in `DIAS Attributes - Values 2017.xlsx`. Therefore, these features will remain in the datasets without any modifications.

### Data Cleaning
For some features listed in `DIAS Attributes - Values 2017.xlsx` **_unknown information_** are encoded to certain numbers. These values will be replaced with `NaN` in `azdias` and `customers` datasets.
1. Features whose information are available in `DIAS Information Levels - Attributes 2017.xlsx` or in `DIAS Attributes - Values 2017.xlsx`, will be selected for the analysis.  
2. The **_unknown information_** will be replaced with `NaN`.  
3. Both `azdias` and `customers` datasets will be compared and features with similar percentage of `NaN` count will be dropped.

In [12]:
# Select the features whose information are available
azdias = azdias.loc[:, features_list]
customers = customers.loc[:, features_list]

In [13]:
# Create a subset of "attributes" with each feature and the associated unknown values
attributes_unknown_val = attributes[(attributes['Meaning'].str.contains("unknown") | attributes['Meaning'].str.contains("no "))]

# Create a list of unknown value for each feature
unknown_val = []
for attribute in attributes_unknown_val['Attribute'].unique():
    val = attributes_unknown_val.loc[attributes['Attribute'] == attribute, 'Value'].astype("str").str.cat(sep=',').split(',')
    val = list(map(int, val)) # Convert the list to "int"
    unknown_val.append(val)
    
# Create a dataframe of features with the list unknown value
attributes_unknown_val = pd.concat([pd.Series(attributes_unknown_val['Attribute'].unique()),
                                    pd.Series(unknown_val)], axis=1)

# Rename the columns
attributes_unknown_val.columns = ['attribute', 'unknown']

# Add a row for ARBEIT and the associated unknown values
#ARBEIT_row = {"attribute" : "ARBEIT", "unknown" : 9}

# Append the row to attributes_unknown_val
attributes_unknown_val = attributes_unknown_val.append({"attribute" : "ARBEIT", "unknown" : [-1, 9]}, ignore_index=True)

In [14]:
%%time
# Replace the unknown values in azdias with NaN based on attributes_unknown_val
for row in attributes_unknown_val.itertuples(index=False):
    if row.attribute in azdias.columns.values.tolist():
        print("Replaceing NaN value on: ", row.attribute)
        nan_val = attributes_unknown_val.loc[attributes_unknown_val['attribute'] == row.attribute, 'unknown'].iloc[0]
        nan_idx = azdias.loc[:, row.attribute].isin(nan_val)
        azdias.loc[nan_idx, row.attribute] = np.NaN
    else:
        continue
        
azdias.head()

Replaceing NaN value on:  AGER_TYP
Replaceing NaN value on:  ALTERSKATEGORIE_GROB
Replaceing NaN value on:  ALTER_HH
Replaceing NaN value on:  ANREDE_KZ
Replaceing NaN value on:  BALLRAUM
Replaceing NaN value on:  BIP_FLAG
Replaceing NaN value on:  CAMEO_DEUG_2015
Replaceing NaN value on:  CAMEO_DEUINTL_2015
Replaceing NaN value on:  CJT_GESAMTTYP
Replaceing NaN value on:  D19_BANKEN_ANZ_12
Replaceing NaN value on:  D19_BANKEN_ANZ_24
Replaceing NaN value on:  D19_BANKEN_DATUM
Replaceing NaN value on:  D19_BANKEN_DIREKT_RZ
Replaceing NaN value on:  D19_BANKEN_GROSS_RZ
Replaceing NaN value on:  D19_BANKEN_LOKAL_RZ
Replaceing NaN value on:  D19_BANKEN_OFFLINE_DATUM
Replaceing NaN value on:  D19_BANKEN_ONLINE_DATUM
Replaceing NaN value on:  D19_BANKEN_ONLINE_QUOTE_12
Replaceing NaN value on:  D19_BANKEN_REST_RZ
Replaceing NaN value on:  D19_BEKLEIDUNG_GEH_RZ
Replaceing NaN value on:  D19_BEKLEIDUNG_REST_RZ
Replaceing NaN value on:  D19_BILDUNG_RZ
Replaceing NaN value on:  D19_BIO_OEKO_RZ
R

Replaceing NaN value on:  KBA13_KRSAQUOT
Replaceing NaN value on:  KBA13_KRSHERST_AUDI_VW
Replaceing NaN value on:  KBA13_KRSHERST_BMW_BENZ
Replaceing NaN value on:  KBA13_KRSHERST_FORD_OPEL
Replaceing NaN value on:  KBA13_KRSSEG_KLEIN
Replaceing NaN value on:  KBA13_KRSSEG_OBER
Replaceing NaN value on:  KBA13_KRSSEG_VAN
Replaceing NaN value on:  KBA13_KRSZUL_NEU
Replaceing NaN value on:  KBA13_KW_30
Replaceing NaN value on:  KBA13_KW_40
Replaceing NaN value on:  KBA13_KW_50
Replaceing NaN value on:  KBA13_KW_60
Replaceing NaN value on:  KBA13_KW_0_60
Replaceing NaN value on:  KBA13_KW_70
Replaceing NaN value on:  KBA13_KW_61_120
Replaceing NaN value on:  KBA13_KW_80
Replaceing NaN value on:  KBA13_KW_90
Replaceing NaN value on:  KBA13_KW_110
Replaceing NaN value on:  KBA13_KW_120
Replaceing NaN value on:  KBA13_KW_121
Replaceing NaN value on:  KBA13_MAZDA
Replaceing NaN value on:  KBA13_MERCEDES
Replaceing NaN value on:  KBA13_MOTOR
Replaceing NaN value on:  KBA13_NISSAN
Replaceing Na

,AGER_TYP,ALTERSKATEGORIE_GROB,ALTER_HH,ANREDE_KZ,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_PERSONEN,ANZ_TITEL,ARBEIT,BALLRAUM,...,SEMIO_VERT,SHOPPER_TYP,SOHO_FLAG,TITEL_KZ,VERS_TYP,WACHSTUMSGEBIET_NB,WOHNDAUER_2008,WOHNLAGE,W_KEIT_KIND_HH,ZABEOTYP
0,NaN,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
1,NaN,1.0,NaN,2.0,11.0,0.0,2.0,0.0,3.0,6.0,...,1.0,3.0,NaN,NaN,2.0,NaN,9.0,4.0,3.0,5.0
2,NaN,3.0,17.0,2.0,10.0,0.0,1.0,0.0,3.0,2.0,...,4.0,2.0,NaN,NaN,1.0,NaN,9.0,2.0,3.0,5.0
3,2.0,4.0,13.0,2.0,1.0,0.0,0.0,0.0,2.0,4.0,...,4.0,1.0,NaN,NaN,1.0,NaN,9.0,7.0,NaN,3.0
4,NaN,3.0,20.0,1.0,3.0,0.0,4.0,0.0,4.0,2.0,...,7.0,2.0,NaN,NaN,2.0,NaN,9.0,3.0,2.0,4.0


In [15]:
%%time
# Replace the unknown values in customers with NaN based on attributes_unknown_val
for row in attributes_unknown_val.itertuples(index=False):
    if row.attribute in customers.columns.values.tolist():
        print("Replaceing NaN value on: ", row.attribute)
        nan_val = attributes_unknown_val.loc[attributes_unknown_val['attribute'] == row.attribute, 'unknown'].iloc[0]
        nan_idx = customers.loc[:, row.attribute].isin(nan_val)
        customers.loc[nan_idx, row.attribute] = np.NaN
    else:
        continue
        
customers.head()

Replaceing NaN value on:  AGER_TYP
Replaceing NaN value on:  ALTERSKATEGORIE_GROB
Replaceing NaN value on:  ALTER_HH
Replaceing NaN value on:  ANREDE_KZ
Replaceing NaN value on:  BALLRAUM
Replaceing NaN value on:  BIP_FLAG
Replaceing NaN value on:  CAMEO_DEUG_2015
Replaceing NaN value on:  CAMEO_DEUINTL_2015
Replaceing NaN value on:  CJT_GESAMTTYP
Replaceing NaN value on:  D19_BANKEN_ANZ_12
Replaceing NaN value on:  D19_BANKEN_ANZ_24
Replaceing NaN value on:  D19_BANKEN_DATUM
Replaceing NaN value on:  D19_BANKEN_DIREKT_RZ
Replaceing NaN value on:  D19_BANKEN_GROSS_RZ
Replaceing NaN value on:  D19_BANKEN_LOKAL_RZ
Replaceing NaN value on:  D19_BANKEN_OFFLINE_DATUM
Replaceing NaN value on:  D19_BANKEN_ONLINE_DATUM
Replaceing NaN value on:  D19_BANKEN_ONLINE_QUOTE_12
Replaceing NaN value on:  D19_BANKEN_REST_RZ
Replaceing NaN value on:  D19_BEKLEIDUNG_GEH_RZ
Replaceing NaN value on:  D19_BEKLEIDUNG_REST_RZ
Replaceing NaN value on:  D19_BILDUNG_RZ
Replaceing NaN value on:  D19_BIO_OEKO_RZ
R

Replaceing NaN value on:  KBA13_KMH_250
Replaceing NaN value on:  KBA13_KMH_251
Replaceing NaN value on:  KBA13_KRSAQUOT
Replaceing NaN value on:  KBA13_KRSHERST_AUDI_VW
Replaceing NaN value on:  KBA13_KRSHERST_BMW_BENZ
Replaceing NaN value on:  KBA13_KRSHERST_FORD_OPEL
Replaceing NaN value on:  KBA13_KRSSEG_KLEIN
Replaceing NaN value on:  KBA13_KRSSEG_OBER
Replaceing NaN value on:  KBA13_KRSSEG_VAN
Replaceing NaN value on:  KBA13_KRSZUL_NEU
Replaceing NaN value on:  KBA13_KW_30
Replaceing NaN value on:  KBA13_KW_40
Replaceing NaN value on:  KBA13_KW_50
Replaceing NaN value on:  KBA13_KW_60
Replaceing NaN value on:  KBA13_KW_0_60
Replaceing NaN value on:  KBA13_KW_70
Replaceing NaN value on:  KBA13_KW_61_120
Replaceing NaN value on:  KBA13_KW_80
Replaceing NaN value on:  KBA13_KW_90
Replaceing NaN value on:  KBA13_KW_110
Replaceing NaN value on:  KBA13_KW_120
Replaceing NaN value on:  KBA13_KW_121
Replaceing NaN value on:  KBA13_MAZDA
Replaceing NaN value on:  KBA13_MERCEDES
Replaceing

,AGER_TYP,ALTERSKATEGORIE_GROB,ALTER_HH,ANREDE_KZ,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_PERSONEN,ANZ_TITEL,ARBEIT,BALLRAUM,...,SEMIO_VERT,SHOPPER_TYP,SOHO_FLAG,TITEL_KZ,VERS_TYP,WACHSTUMSGEBIET_NB,WOHNDAUER_2008,WOHNLAGE,W_KEIT_KIND_HH,ZABEOTYP
0,2.0,4.0,10.0,1.0,1.0,0.0,2.0,0.0,1.0,3.0,...,6.0,3.0,NaN,NaN,1.0,NaN,9.0,7.0,6.0,3.0
1,NaN,4.0,11.0,1.0,NaN,NaN,3.0,0.0,NaN,NaN,...,7.0,3.0,NaN,NaN,1.0,NaN,9.0,NaN,NaN,3.0
2,NaN,4.0,6.0,2.0,1.0,0.0,1.0,0.0,3.0,7.0,...,3.0,1.0,NaN,NaN,2.0,NaN,9.0,2.0,6.0,3.0
3,1.0,4.0,8.0,1.0,0.0,NaN,0.0,0.0,1.0,7.0,...,7.0,0.0,NaN,NaN,1.0,NaN,9.0,7.0,NaN,1.0
4,NaN,3.0,20.0,1.0,7.0,0.0,4.0,0.0,3.0,3.0,...,5.0,1.0,NaN,NaN,2.0,NaN,9.0,3.0,2.0,1.0
